In [1]:
import pyspark 
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('DataFrame').getOrCreate()

In [2]:
columns = ['currency', 'value']
inputdata = [('Euro', 90), ('Pound', 100), ('Yuan', 11), ('Yen', 2), ('US Dollar', 74), ('K Dinar', 242)]

# RDD
rdd = spark.sparkContext.parallelize(inputdata)
rddDF = rdd.toDF()
df = rddDF.withColumnRenamed('_1', 'Currency')
df.show()
df = spark.createDataFrame(rdd).toDF(*columns)
df.show()

+---------+---+
| Currency| _2|
+---------+---+
|     Euro| 90|
|    Pound|100|
|     Yuan| 11|
|      Yen|  2|
|US Dollar| 74|
|  K Dinar|242|
+---------+---+

+---------+-----+
| currency|value|
+---------+-----+
|     Euro|   90|
|    Pound|  100|
|     Yuan|   11|
|      Yen|    2|
|US Dollar|   74|
|  K Dinar|  242|
+---------+-----+



In [3]:
df = spark.createDataFrame(data=inputdata, schema=columns)
df.show()

+---------+-----+
| currency|value|
+---------+-----+
|     Euro|   90|
|    Pound|  100|
|     Yuan|   11|
|      Yen|    2|
|US Dollar|   74|
|  K Dinar|  242|
+---------+-----+



In [14]:
df.write.format('csv').save('test1/')

In [13]:
# write without partition
df.repartition(1).write.format('csv').save('test/', header=True)

In [11]:
# write to text will not work from dataframe if more than one columns, so convert to single column and save then save to text file.
df.repartition(1).write.format('text').save('test/')

AnalysisException: path file:/home/girish/Documents/mahe/Lectures/Deeapk sir- ABD/spark/Class Programs/08Jan2020/test already exists.;

In [18]:
df.rdd.map(lambda x: x[0] + "," + str(x)).repartition(1).saveAsTextFile('Text')

In [19]:
dfCSV = spark.read.csv('path of csv', header=True)

AnalysisException: Path does not exist: file:/home/girish/Documents/mahe/Lectures/Deeapk sir- ABD/spark/Class Programs/08Jan2020/path of csv;

In [ ]:
dfmysql = spark.read.format('jdbc')\
             .option('url', 'jdbc:mysql://ipaddress or url/retail_db')\
             .option('driver', )

In [ ]:
df = spark.read.format('csv')\
     .option('header', 'true')\
     .option('inferSchema', 'true')\
     .load('path of csv')
df.show()


In [ ]:
from pyspark.sql.functions import col, expr, column, udf, date_sub, date_add, col, datediff
from pyspark.sql.types import StringType, IntegerType

df.select(col('DEST_COUNTRT_NAME')).show(5, False)
df.select(column('DEST_COUNTRT_NAME')).show(5, False)
df.select('DEST_COUNTRT_NAME').show(5, False)
df.select(col('DEST_COUNTRT_NAME', 'ORIGIN_COUNTRY_NAME')).show(5, False)
df.select(expr('DEST_COUNTRT_NAME as Destionation')).show(5, False)
# Creates new dataframe for all results

In [ ]:
# add column
df.withColumn('withinCountry', expr('ORIGIN_COUNTRY_NAME == DEST_COUNTRY'))

In [ ]:
# Adding column by user defined functions

def computeGroup(count):
    if count < 2:
        return 'Min'
    else:
        return 'Busy'
    
group_udf = udf(computeGroup, StringType())
df.withColumn('Frequency', group_udf(col('count'))).show(5, False)